In [ ]:
!pip install openai

In [ ]:
!sudo apt-get install unzip
!unzip /content/cyberbullying_data_clean.zip

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import random
import pickle

**CAREFUL WITH API KEY!**

In [ ]:
import base64
import requests

# OpenAI API Key
api_key = "put your api key here"

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}


In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


def response_text_to_label(response_text):
  if 'a.' in response_text:
    return 1

  if 'b.' in response_text:
    return 0

  return -1 #uncertain

In [ ]:
dataset_path = 'cyberbullying_data_clean'
cyberbullying_path = dataset_path + '/cyberbullying'
non_cyberbullying_path = dataset_path + '/non_cyberbullying'

cyberbullying_images = os.listdir(cyberbullying_path)
non_cyberbullying_images = os.listdir(non_cyberbullying_path)

prompt1 = 'Cyberbullying content is used to harass, threaten, embarrass, or target another person.  Below are 10 images that contain cyberbullying content.  Create a list of 3 criteria based on the features in the cyberbullying images that can be used to determine whether an image contains cyberbullying content.  The criteria should be framed as short, concise one-sentence questions and numbered 1, 2, and 3.'
prompt2 = 'Cyberbullying content is used to harass, threaten, embarrass, or target another person.  Below are 10 images.  The first 5 contain cyberbullying content, and the last 5 do not contain cyberbullying content.  Create a list of 3 criteria based on the features in the cyberbullying images that can be used to determine whether an image contains cyberbullying content.  The criteria should be framed as one-sentence questions and numbered 1, 2, and 3.'



In [ ]:
if input('Sure? Y/N ') == 'Y':

  cyberbullying_subset = random.sample(cyberbullying_images, 1000)
  non_cyberbullying_subset = random.sample(non_cyberbullying_images, 1000)

  with open('cyberbullying_subset_exp.pkl', 'wb') as f:
    pickle.dump(cyberbullying_subset, f)

  with open('non_cyberbullying_subset_exp.pkl', 'wb') as f:
    pickle.dump(non_cyberbullying_subset, f)

In [ ]:
cyberbullying_images = pickle.load(open('cyberbullying_subset_exp.pkl', 'rb'))
non_cyberbullying_images = pickle.load(open('non_cyberbullying_subset_exp.pkl', 'rb'))

In [ ]:
few_shot_cyberbullying_images = ['1.anti-rainbow-flag-750x_0.jpg', '7.s-s-unshaven-sad-ashamed-man-doing-loser-sign-hand-fingers-his-front-funny-depressed-face-expression-s-139158713.jpg', '11.500_F_210144311_9SCWDyIi9SWOStydvAdmF2x89bwzrgQr.jpg', '12.Pg-42-sharp1-sandinson.jpg', '19.noose-around-neck-drawing-19.jpg']
few_shot_non_cyberbullying_images = ['27770950@N02_14364532144.jpg', '24021901@N00_1040628046.jpg', '40683746@N06_3744043658.jpg', '42002724@N07_15783744428.jpg', '256616725152145519_21816927.jpg']

groups = [random.sample(cyberbullying_subset, 10) for i in range(100)]

def make_payload(groups):
  payload = {
        "model": "gpt-4-turbo",
        "messages": [
          {
            "role": "user",
            "content": [

            ]
          }
        ],
        "max_tokens": 300
      }
  payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': prompt1
        }
    )
  for image in groups:
    image_path = f'{cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload['messages'][0]['content'].append(
        {
            'type': 'image_url',
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              'detail': 'low'
              }
        }
    )
    """payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': 'The above image contains cyberbullying content'
        }
    )"""

  """for image in few_shot_non_cyberbullying_images:
    image_path = f'{non_cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload['messages'][0]['content'].append(
        {
            'type': 'image_url',
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              'detail': 'low'
              }
        }
    )
    payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': 'The above image does not contain cyberbullying content'
        }
    )"""

  payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': 'Criteria (in a numbered list): '
        }
    )

  return payload

In [ ]:
payload = make_payload(groups[99])

In [ ]:
payload

In [ ]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [ ]:
response.json()['choices'][0]['message']['content']

In [ ]:
criteria = []

for group in tqdm(groups):
  payload = make_payload(group)
  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  text = response.json()['choices'][0]['message']['content']
  criteria.append(text)
  with open('criteria.pkl', 'wb') as f:
    pickle.dump(criteria, f)

  if random.random() < 0.1:
    print(text)

In [ ]:
processed_criteria = []
for item in criteria:
  for i in range(len(item) - 1):
    if item[i] == '1' and item[i + 1] == '.':
      pos_1 = i

    if item[i] == '2' and item[i + 1] == '.':
      pos_2 = i

    if item[i] == '3' and item[i + 1] == '.':
      pos_3 = i

  processed_criteria.append(item[pos_1 + 3:pos_2])
  processed_criteria.append(item[pos_2 + 3:pos_3])
  processed_criteria.append(item[pos_3 + 3:])

In [ ]:
processed_criteria = ['**Does the image depict or imply physical violence, intimidation, or threats?**  \n   This checks if the content showcases actions or gestures that could be perceived as threatening or harmful.\n\n',
 '**Does the image use offensive or derogatory language, or imagery meant to demean or ridicule someone?**  \n   This assesses if text overlays, displayed gestures, or facial expressions are used to explicitly humiliate or insult individuals.\n\n',
 '**Are there signs of targeting an individual or a specific group for harm or ridicule?**  \n   This evaluates the context or elements in the image that may suggest a particular person or group is being singled out negatively.',
 'Does the image include gestures or symbols commonly associated with offensive or threatening meanings?\n',
 'Are there text overlays or visual elements in the image that convey harmful, menacing, or degrading messages?\n',
 'Is the context or setting of the image suggesting a scenario meant to embarrass, frighten, or demean another individual?',
 '**Does the image include any aggressive or offensive gestures that are intended to demean or insult an individual or group?**\n\n',
 '**Does the image contain abusive or derogatory text or captions targeting specific individuals or groups based on their identity or personal attributes?**\n\n',
 '**Does the image portray or suggest harm or threat towards an individual or group in a manner that could incite fear, distress, or anxiety?**',
 'Does the image include symbols, gestures, or expressions traditionally associated with aggression, offence, or disrespect?\n',
 'Does the image show displays of weapons or suggest violent actions which may be used to intimidate or threaten?\n',
 'Does the imagery include any written or visual content implying harm, ridicule, or danger to individuals or groups?',
 '**Does the image contain gestures or expressions commonly associated with hostility or disrespect, such as showing the middle finger or making a thumbs down gesture?**\n',
 '**Are there any text overlays or captions that include threatening, insulting, or degrading language aimed at an individual or group?**\n',
 '**Does the context or setting of the image suggest it was shared to humiliate, mock, or belittle someone specifically?**',
 'Does the image contain any gestures, symbols, or text that could be perceived as offensive or harmful to a particular individual or group?\n',
 'Is there any depiction or suggestion of violence, self-harm, or threatening behavior targeted at an individual or group?\n',
 'Does the context or caption of the image encourage hostility, ridicule, or discrimination against someone based on their identity, appearance, or beliefs?',
 'Does the image show someone being threatened, intimidated, or depicted in a demeaning manner?\n',
 'Are there any symbols, gestures, or text used in the image that could be considered offensive or harassing?\n',
 'Is the context or setting of the image suggestive of targeting a specific individual or group in a negative way?',
 'Does the image include gestures or symbols commonly associated with hate or aggression?\n',
 'Are there any text or visual elements in the image that suggest harm or ridicule towards a specific individual or group?\n',
 'Does the image indirectly or directly promote actions or expressions of violence or discrimination against someone based on their identity or personal attributes?',
 'Does the image depict or imply threat, aggression, or use of weapons towards an individual or group?\n',
 'Are there any derogatory symbols, gestures, or expressions targeting a specific person or group of people?\n',
 'Is the context or caption of the image suggestive of harassment, humiliation, or intimidation of an individual?',
 'Does the image contain symbols or gestures associated with violence, intimidation, or self-harm?\n',
 'Is there an identifiable use of aggressive or threatening language in any text accompanying the image?\n',
 'Does the image specifically seem intended to ridicule, shame, or demean a particular individual or group?',
 'Does the image include gestures or expressions that are commonly associated with disrespect, obscenity, or aggression?\n',
 'Is the context or setting of the image suggestive of ridicule, mockery, or targeted humiliation towards individuals or groups?\n',
 'Are there any text overlays, captions, or symbols that might promote hostility, threats, or bullying behavior?',
 'Does the image contain aggressive gestures or symbols that are universally recognized as offensive or threatening?\n',
 'Is anyone in the image displaying weapons in a manner that could be perceived as threatening or intending to incite violence?\n',
 'Does the image seem to target or demean an individual either visually or through accompanying text?',
 'Does the image depict any symbols, gestures, or expressions that could reasonably be interpreted as offensive or threatening to specific groups or individuals?\n',
 'Is there any inclusion of content such as weapons, harmful objects, or imagery suggesting physical violence or intimidation?\n',
 'Are there textual elements, overlays, or context within the image that mock, disparage, or incite negativity or harm towards an individual or group?',
 'Does the image contain symbols, gestures, or text that might be considered offensive or derogatory toward a particular individual or group?\n',
 'Is the image or its caption intended to intimidate, threaten, or humiliate someone based on their identity, actions, or beliefs?\n',
 'Does the content explicitly or implicitly encourage discriminatory attitudes or actions against a specific group or individual?',
 'Does the image contain symbols or gestures commonly associated with aggression or insults?\n',
 'Is the content of the image intended to intimidate, such as displaying weapons or threatening poses?\n',
 'Does the image depict any form of self-harm or harm towards others that could be used to distress or manipulate someone?',
 '**Does the image include threatening gestures or symbols directed toward an unspecified viewer or a specific group?**\n',
 '**Are there any texts or visual elements within the image that express humiliation, intimidation, or aggression?**\n',
 '**Does the context or composition of the image seem to isolate or target an individual or group in a derogatory or demeaning manner?**',
 'Does the image depict or imply harm or threats towards an individual or a group?\n',
 'Are symbols of hate or exclusion prominently featured in the image?\n',
 'Does the content of the image aim to ridicule, mock, or demean someone based on their identity or personal attributes?',
 'Does the image feature symbols or gestures commonly associated with offensive or threatening messages?\n',
 'Are there elements in the image that imply physical harm or display weapons in a context that could intimidate or threaten others?\n',
 'Does the image include text or visual content that could be interpreted as ridiculing, shaming, or attacking a specific individual or group?',
 'Does the image include gestures, symbols, or text that are commonly associated with threats, offensive behavior, or derogation?\n',
 'Is the image manipulated to ridicule, embarrass, or portray someone in a harmful or derogatory way?\n',
 'Are there any captions or overlaid text in the image that contain harmful, threatening, or explicit language directed at an individual or group?',
 'Does the image include gestures or symbols that are widely recognized as offensive or aggressive?\n',
 'Are there elements in the image such as text or visual cues that directly threaten, intimidate, or humiliate a person?\n',
 'Does the context or accompanying text of the image target an individual or group in a derogatory or demeaning manner?',
 'Does the image contain explicit threats of violence or allusions to self-harm or harm to others?\n',
 'Does the image explicitly or implicitly involve derogatory or humiliating language targeting an individual or group?\n',
 'Does the image depict offensive gestures or symbols that are generally recognized as insulting or demeaning?',
 'Does the image display any forms of aggressive or threatening gestures or symbols, such as middle fingers, guns pointed towards the viewer, or depictions of suicide?\n',
 'Is the image accompanied by text or symbols that could be interpreted as harmful, insulting, or intended to shame or degrade an individual or group?\n',
 'Does the image involve inappropriate or mocking mimicry, such as pretending to be injured or wearing offensive masks, aimed at ridiculing or taunting someone?',
 '**Does the image contain symbols or gestures known to be offensive or aggressive?**  \n   For example, the use of middle fingers, violent poses, or representation of harmful items.\n\n',
 '**Does the image include language or imagery created to intimidate, humiliate, or demean a person or group?**  \n   This could be visual text or edited features meant to ridicule or mock.\n\n',
 '**Is the context of the image suggesting a threat or promoting harm against individuals or a group?**  \n   This could include depictions of weapons aimed at the viewer, or creation of threatening scenarios.',
 'Does the image contain gestures or symbols that are universally recognized as offensive or threatening?\n',
 'Is there any imagery or text that could be interpreted as promoting harm or violence towards an individual or group?\n',
 'Are there elements in the image that seem to demean or humiliate a person based on their appearance, situation, or perceived weaknesses?',
 'Does the image include symbols, gestures, or text that could be interpreted as threatening or offensive to an individual or group?\n',
 'Is the image portraying or implying violence, self-harm, or harmful actions toward others directly or indirectly?\n',
 'Does the content of the image target a specific individual, either by including their likeness or personal identifiers in a context that could be seen as harassing or demeaning?',
 'Does the image display aggressive gestures, provocative poses, or symbols commonly associated with negativity or hostility?\n',
 'Does the image contain elements such as weapons, offensive gestures, or imagery that could be used to intimidate or threaten a person?\n',
 'Is there any text or symbols added to the image that include insults, threats, or derogatory language targeting an individual or group?',
 'Does the image include gestures or symbols that can be interpreted as threatening or offensive?\n',
 'Is the image being used in a context that aims to intimidate, harass, or embarrass an individual or group?\n',
 'Are there any textual elements or modifications in the image that contribute to a hostile or demeaning environment?',
 'Does the image feature symbols or gestures commonly associated with hate, harm, or threats towards individuals or groups?\n',
 'Is the content of the image aimed at publicly shaming or humiliating someone directly or indirectly?\n',
 'Does the image use violent or aggressive imagery to convey a threatening message towards a person or community?',
 '**Does the image depict offensive or threatening gestures towards specific individuals or groups?**\n',
 '**Does the image contain text or symbols that are commonly associated with harassment, insults, or threats?**\n',
 '**Are there indications that the context of the image is meant to ridicule, shame, or humiliate someone publicly?**',
 'Does the image depict or imply threatening, aggressive, or violent behavior towards a person or group?\n',
 'Does the image include gestures or expressions that are commonly understood to be disrespectful, mocking, or offensive?\n',
 'Are there any text overlays, captions, or context provided with the image that ridicule, demean, or harass individuals depicted in the image?',
 'Does the image depict any forms of threats or implied violence toward specific individuals or groups?\n',
 'Are there any explicit or implicit gestures or symbols that could be interpreted as offensive or disrespectful?\n',
 'Does the image include any text or symbols that promote hostility or intimidation?',
 'Does the image contain symbols, gestures, or expressions that can be interpreted as offensive or threatening?\n',
 'Is there any text or imagery in the image that directly or indirectly promotes exclusion, hatred, or discrimination against specific groups or individuals?\n',
 'Are aggressive or hostile actions or behaviors displayed in the image towards another person or group?',
 'Does the image display messages, symbols, or gestures that are commonly associated with hate, aggression, or humiliation?\n',
 'Is the image altered to degrade, mock, or falsely portray the individual(s) in a negative or harmful manner?\n',
 'Is the image accompanied by text or context that encourages harmful behavior or threats towards the individual(s) depicted?',
 'Does the image contain symbols, gestures, or texts that explicitly or implicitly degrade, insult, or offend individuals or groups?\n',
 'Is there any visible content that promotes hostility or violence against a specific individual or group based on race, ethnicity, gender, sexuality, or other personal characteristics?\n',
 'Does the imagery indulge in mocking, threatening, or manipulating the depicted individuals, potentially causing harm or distress?',
 'Does the image include symbols, gestures, or text that could be interpreted as aggressive or offensive towards individuals or groups?\n',
 'Is there any depiction or suggestion of harm, exclusion, or derogation towards a specific individual or group?\n',
 'Are there any elements in the image that specifically target, mock, or belittle someone based on their identity such as race, gender, sexuality, or personal beliefs?',
 'Does the image show any gestures or symbols that are generally understood as offensive or aggressive?\n',
 'Is the subject in the picture depicted in a way that could be interpreted as mocking, insulting, or demeaning?\n',
 'Are there any text or visual elements that suggest targeting or harassing an individual or group?',
 'Does the image contain gestures or expressions generally associated with aggression or hostility (e.g., making rude gestures, displaying weapons in a threatening manner)?\n',
 'Is the image accompanied by text or captions that demean, ridicule, or threaten an individual or group?\n',
 "Does the context of the image suggest it is being used to embarrass, intimidate, or harm someone's reputation or mental well-being?",
 'Does the image contain gestures or symbols commonly associated with offensive or threatening messages (e.g., aggressive hand gestures)?\n',
 'Does the image include any text or captions that express hostility, threats, shaming, or derogatory terms directed at individuals or groups?\n',
 'Are there elements in the image that can be perceived as promoting harmful or dangerous behavior towards oneself or others?',
 '**Does the image include symbols, gestures, or expressions that are commonly associated with offensive or threatening behavior?**\n   - Examples include making a rude gesture, displaying a weapon in a threatening manner, or expressions of disdain or aggression.\n\n',
 '**Does the context or caption of the image suggest intent to ridicule, embarrass, or demean a specific individual or group?**\n   - Contextual clues might include sarcastic or derogatory captions, tags, or comments that target someone based on personal attributes or identity.\n\n',
 '**Is the image manipulated or edited in a way that could humiliate or distort the representation of the subjects depicted?**\n   - This includes photoshopped images that distort physical appearances or context to create a misleading or demeaning portrayal.',
 'Does the image include gestures or facial expressions that could be intended to intimidate, threaten, or mock someone?\n',
 'Are there symbols, signs, or text in the image that could be interpreted as promoting violence or discrimination against specific individuals or groups?\n',
 'Is the content of the image being used out of its original context to ridicule or demean someone?',
 'Does the image include gestures or expressions commonly associated with disrespect or hostility (e.g., middle finger, thumbs down)?\n',
 'Is the content of the image intended to demean, insult, or intimidate an individual or group?\n',
 'Does the image include textual elements that contain taunting, abusive, or threatening language?',
 'Does the image contain gestures or expressions that are commonly associated with disrespect or hostility, such as middle fingers, sticking out tongues, or mockingly sad faces?\n',
 'Are there any indications that the person in the image is being depicted in a compromising, embarrassing, or harmful scenario, either through the way they are posing or the context provided by the image?\n',
 "Does the image include text, symbols, or other elements that might suggest it's being used to mock, threaten, or demean someone, especially if shared publicly or within a group?",
 'Does the image depict or imply threats, aggression, or violence towards an individual or group?\n',
 'Are there visual elements or text in the image that could be used to embarrass, humiliate, or demean someone?\n',
 'Does the context of the image suggest it is being shared to intentionally provoke, harass, or cause distress to others?',
 'Does the image contain gestures or expressions that are commonly associated with insult or offense, such as obscene hand gestures?\n',
 'Is the text accompanying the image using language that is derogatory, threatening, or intended to shame or embarrass an individual?\n',
 'Does the context or setting of the image suggest a purpose to humiliate or provoke aggression towards an individual or a group?',
 'Does the image depict or imply any gestures or symbols that are widely recognized as offensive or threatening?\n',
 'Is there any text or visible context in the image that could be construed as harassing, threatening, or meant to publicly embarrass an individual or group?\n',
 'Does the image show any form of aggressive posturing or use of weapons in a manner that could be perceived as intimidating or coercive?',
 '**Is the image used to ridicule or mock individuals based on their appearance, actions, or perceived characteristics?**\n   - Evaluates if the content is demeaning or derogatory toward a person or group.\n\n',
 '**Does the image include symbols, gestures, or text that convey threats, hate, or promote violence?**\n   - Assess if there are threatening elements, such as aggressive gestures or symbols known to be associated with hate speech or violence.\n\n',
 '**Are there any elements in the image that promote harmful stereotypes or offensive assumptions about a group of people?**\n   - Looks for content that reinforces negative societal stereotypes or offensive assumptions about age, gender, race, or other characteristics.',
 'Does the image include gestures or symbols that could be perceived as threatening or offensive?\n',
 'Is the context or caption of the image intended to ridicule, threaten, or demean a specific individual or group?\n',
 'Does the image include any altered or edited elements that exaggerate physical features or attributes to humiliate or embarrass someone?',
 'Does the image contain symbols, gestures, or text that could be perceived as threatening or promoting harm towards specific individuals or groups?\n',
 'Is there any content in the image that seems to ridicule, mock, or disparage individuals or specific groups based on identity factors such as race, gender, sexuality, or others?\n',
 'Are there elements in the image that are designed to embarrass, intimidate, or exert power over others, such as derogatory gestures or explicit negative expressions directed towards the camera or visible target?',
 '**Does the image depict or imply violence, threats, or harm towards an individual or group?**\n',
 '**Does the image contain gestures or expressions that are commonly associated with disrespect, ridicule, or offense?**\n',
 '**Is the content of the image likely to intimidate, exclude, or stigmatize an individual based on personal characteristics or identity?**',
 'Does the image contain gestures or symbols that are widely recognized as offensive or derogatory?\n',
 'Is the person in the image depicted in a way that could be seen as threatening, intimidating, or harmful to their dignity?\n',
 'Does the image include text or imagery that belittles, mocks, or demeans another individual or group based on personal characteristics?',
 'Does the image display gestures or expressions commonly associated with offensive or disrespectful behavior?\n',
 'Are there any threatening props or symbols, such as weapons or aggressive postures, that suggest intimidation or harm?\n',
 'Does the content of the image involve demeaning or derogatory text or slogans directed at an individual or group?',
 'Does the image display content that might be intended to intimidate, threaten, or invoke fear in someone (e.g., depiction of weapons aimed at the viewer)?\n',
 'Does the image depict expressions or gestures that are commonly associated with mockery, ridicule, or demeaning attitudes towards an individual or group (e.g., thumbs down, making faces)?\n',
 'Are there any elements or symbols in the image that are often associated with hateful, harmful, or violent messages (e.g., nooses or other aggressive symbols)?\n\nThese criteria focus on visual cues and content that might imply harassment, intimidation, or ridicule, which are common in cyberbullying imagery.',
 'Does the image include gestures or symbols that are universally recognized as offensive or threatening?\n',
 'Are there elements in the image that could be interpreted as promoting violence or harm towards an individual or group?\n',
 'Is the image shared or used in a context that seems intended to embarrass, harass, or humiliate someone specifically?',
 '**Is the image being used to intimidate, threaten, or cause fear?** (E.g., images that include weapons being pointed or threatening gestures directed at the viewer.)\n   \n',
 '**Does the image depict any gestures or expressions that could be interpreted as mocking, derogatory, or offensive?** (E.g., disrespectful hand gestures, facial expressions that mimic or ridicule, or inclusion of symbolic gestures known to be offensive.)\n\n',
 '**Is the image accompanied by text or context that explicitly targets or harasses an individual or group?** (E.g., captions that belittle, threaten, or single out someone for abuse or ridicule.)',
 'Does the image contain any gestures or symbols commonly associated with offensive or disrespectful behavior?\n',
 'Are there objects, text, or imagery in the image that can be perceived as threatening or promoting harm?\n',
 'Does the context or the composition of the image appear to intentionally demean, mock, or belittle a person or group?',
 'Does the image contain any threatening or violent symbolism, such as displayed weapons or gestures implying harm?\n',
 'Are there any explicit or implicit messages or gestures in the image that might be used to ridicule, shame, or degrade an individual or group?\n',
 'Does the image contain any signs of exclusion or mockery, such as derogatory gestures or expressions targeted at an individual?',
 'Does the image include symbols, gestures, or explicit content that could be considered threatening or offensive?\n',
 'Does the image depict any form of aggressive behavior or hostility towards a person or a group of people?\n',
 'Does the content of the image seem intended to shame, bully, or humiliate someone specifically?',
 'Does the image include gestures or symbols that are commonly recognized as offensive or provocative?\n',
 'Are there elements in the image that implicitly or explicitly express threats or intimidation?\n',
 'Is the subject depicted in a manner that is demeaning or meant to publicly shame or humiliate them?',
 '**Does the image contain gestures or symbols that are commonly associated with offensive or threatening behavior?**\n',
 '**Is the image accompanied by text, captions, or comments that may indicate harassment, shaming, or intimidation?**\n',
 '**Does the context or setting of the image suggest it is being used to embarrass, target, or cause distress to an individual?**',
 'Does the image include symbols, gestures, or text that could be interpreted as offensive or threatening towards individuals or groups?\n',
 'Is there any inclusion of derogatory language or images aimed at ridiculing, shaming, or demeaning others based on identity, appearance, or beliefs?\n',
 'Does the content of the image promote or depict violence, intimidation, or the exclusion of an individual or group?',
 'Does the image include any aggressive gestures or symbols that could be interpreted as threatening or demeaning?\n',
 'Does the content of the image carry text or symbols that have historically or contextually been used to harass or insult individuals?\n',
 'Are there any elements in the image that seem to specifically target, mock, or degrade an individual or group based on identity, appearance, or characteristics?',
 '**Does the image contain symbols, gestures, or text that are widely recognized as offensive or harmful?**\n   - Example: Use of offensive gestures, hate symbols, or derogatory language.\n\n',
 '**Is the image targeting or mocking a specific group or individual based on their identity?**\n   - Example: Images that are derogatory towards a race, gender, ethnicity, religion, or personal characteristics.\n\n',
 '**Does the image display threats, intimidation, or promote violence against a person or group?**\n   - Example: Displaying weapons in a menacing manner or captions that threaten physical harm.',
 '**Does the image contain aggressive or threatening gestures such as pointing a weapon or displaying an offensive hand gesture?**\n',
 '**Is there textual or visual content intended to intimidate, harass, or mock a specific individual or group?**\n',
 '**Are there symbols, signs, or actions in the image that are widely recognized as disrespectful or hostile?**',
 'Does the image contain gestures or symbols that are commonly associated with offensive or threatening behavior?\n',
 'Are there visual elements in the image that suggest harm or violence towards oneself or others?\n',
 'Does the portrayal in the image intentionally demean, mock, or provoke a specific group or individual based on identity factors?',
 'Does the image include gestures or expressions that are commonly associated with disrespect or hostility (such as showing the middle finger or aggressive facial expressions)?\n',
 'Does the context or composition of the image suggest a portrayal of violence or the threat of harm (like displaying weapons pointed at the viewer)?\n',
 'Are there elements in the image that could be interpreted as intentionally demeaning, ridiculing, or shaming a person or group?',
 'Does the image show actions or symbols that are commonly associated with insulting or demeaning gestures?\n',
 'Does the image contain hateful symbols, signs, or text aimed at a particular group based on race, gender, ethnicity, or sexual orientation?\n',
 'Is the image clearly designed to intimidate, threaten, or provoke fear in individuals or specific groups?',
 'Does the image include symbols, gestures, or content commonly associated with violence, threats, or intimidation?\n',
 'Are there elements in the image that suggest disparagement, ridicule, or humiliation of an individual or group?\n',
 "Is there any use of derogatory or inflammatory language, signs, or symbols in the image that could be used to harm someone's reputation or mental well-being?",
 'Does the image feature or imply any form of threat, intimidation, or violence towards an individual or group?\n',
 'Does the image display any symbols, gestures, or messages generally recognized as offensive or hateful?\n',
 'Are there elements in the image that target the dignity or self-esteem of a person, possibly leading to public embarrassment or humiliation?',
 'Does the image contain any gestures or symbols that are widely regarded as offensive or threatening?\n',
 'Is there any text or imagery that could be interpreted as a direct threat or incitement of violence toward an individual or group?\n',
 'Does the context or composition of the image exploit or demean any individual based on personal characteristics or identity attributes?',
 'Does the image contain symbols, gestures, or text that could be interpreted as threatening or promoting harm towards individuals?\n',
 'Are there elements in the image that could be used to shame, embarrass, or negatively stereotype someone based on their appearance, identity, or beliefs?\n',
 'Does the image depict individuals or scenes in a context that could be seen as encouraging exclusion, mockery, or humiliation of someone for the amusement of others?',
 '**Is the image depicting or promoting harm or violence towards an individual or a group?**\n   - This includes images showing people with weapons in a threatening manner or implying physical harm.\n\n',
 '**Does the image contain gestures or symbols that are universally recognized as offensive or derogatory?**\n   - Look for gestures that are culturally acknowledged as insulting or demeaning.\n\n',
 '**Is there any text or context in the image that suggests a demeaning, taunting, or threatening message?**\n   - Consider whether captions, tags, or visible text add a layer of harassment or mockery directed at a person or group.',
 'Does the image contain symbols, gestures, or expressions commonly associated with hate, rejection, or offense?\n',
 'Is there any discriminatory or derogatory symbolism targeting specific groups based on race, gender, sexual orientation, or other identity markers?\n',
 'Does the context or accompanying text of the image explicitly or implicitly threaten, harass, or humiliate individuals or groups?',
 'Does the image contain gestures or symbols that are universally recognized as offensive or provocative?\n',
 'Is there any element in the image that implies a threat, harm, or violence towards an individual or group?\n',
 'Does the content of the image aim to humiliate, mock, or demean someone based on personal characteristics or identity?',
 'Does the image include gestures or symbols that are generally recognized as offensive or threatening?\n',
 'Is there text or imagery included that could be interpreted as promoting harm or displaying hate towards specific individuals or groups?\n',
 'Are there any elements in the image that implicitly or explicitly intimidate, mock, or degrade someone based on personal attributes such as appearance, beliefs, or behaviors?',
 'Does the image include gestures or expressions commonly associated with aggression or disrespect (e.g., showing the middle finger or threatening poses)?\n',
 'Does the context or accompanying text suggest the image is being used to embarrass, threaten, or belittle another person?\n',
 'Are symbols or edited elements present that target or label individuals in a derogatory or harmful manner?',
 '**Does the image depict aggressive gestures or expressions aimed directly at the viewer or another identifiable subject in the image?**\n   - This question helps identify if there are overt signs of aggression or intimidation such as showing weapons or making threatening gestures.\n\n',
 '**Are there any elements in the image that suggest ridicule, mockery, or disparagement either through visual cues or text?**\n   - This includes facial expressions or hand gestures that are commonly used to mock or dismiss someone (e.g., thumbs down or mimicking offensive gestures).\n\n',
 '**Is there any implied or explicit threat of harm or violence in the image?**\n   - This relates to the display of weapons or notional threats directed at the viewer or a person not visible within the image.',
 'Does the image contain symbols or gestures known to be offensive or used to demean others?\n',
 'Does the text in the image contain threats, harmful stereotypes, or derogatory language aimed at individuals or groups?\n',
 'Does the image depict or encourage harmful behavior, including self-harm or harm to others, either literally or symbolically?',
 'Does the image depict gestures or symbols commonly associated with disrespect or aggression towards others?\n',
 "Are there elements in the image that might be used to intimidate, threaten, or harm an individual's reputation?\n",
 'Does the content of the image contain any signs of targeting a specific person or group with negative, harmful, or derogatory messages?',
 'Does the image include any gestures or symbols commonly associated with offensive or threatening meanings?\n',
 'Is the image paired with text or context that suggests the content is being used to embarrass, threaten, or harass an individual or group?\n',
 'Are there visual cues, such as the inclusion of weapons or potentially harmful objects, that could be interpreted as threatening or promoting harm?',
 'Does the image contain symbols, gestures, or text that could be interpreted as threatening or promoting harm towards an individual or group?\n',
 'Does the image depict or imply violence, harm, or aggressive actions towards oneself or others?\n',
 'Is the image used in a context that aims to ridicule, shame, or maliciously stereotype an individual or group based on their identity (e.g., race, gender, sexuality)?',
 'Does the image feature gestures or expressions commonly associated with hostility or disrespect, such as middle fingers or aggressive pointing?\n',
 'Is there any text or symbolic representation in the image that could be interpreted as threatening, demeaning, or derogatory?\n',
 'Does the context or setting of the image suggest the subject is being portrayed in a manner intended to embarrass, intimidate, or ridicule?',
 'Does the image contain any gestures, symbols, or texts that could be interpreted as offensive or threatening?\n',
 'Is anyone in the image portrayed in a demeaning or ridiculing manner that could humiliate or emotionally harm an individual?\n',
 'Are there elements in the image that suggest the promotion of violence or harm towards individuals or groups based on their identity or characteristics?',
 'Does the image depict or advocate violence or threats towards an individual or group?\n',
 'Is aggressive, insulting, or derogatory language (visual or textual) used in the image to demean or intimidate someone?\n',
 'Does the image use symbols, gestures, or actions that are commonly understood to be offensive or hostile?',
 'Does the image portray a gesture or symbol commonly associated with aggressive, obscene, or threatening behavior?\n',
 'Is there an implication or explicit display of violence towards an individual or group in the image?\n',
 'Does the text or context accompanying the image serve to harass, demean, or embarrass a specific person or group?',
 'Does the image depict or insinuate violence, intimidation, or threats towards an individual or a group?\n',
 'Does the image include text or imagery that could be perceived as derogatory, offensive, or humiliating?\n',
 'Is the context or the usage of the image likely to cause emotional distress, fear, or harm to the person(s) depicted?',
 'Does the image contain visual symbols or gestures historically or culturally associated with offensive, threatening, or derogatory connotations?\n',
 'Does the image depict any form of physical or symbolic aggression intended to humiliate or demean a specific individual or group?\n',
 'Does the image promote or denote harm, either physical or psychological, towards someone specific or to people in general?',
 '**Does the image display symbols or gestures commonly associated with threats or harm?** (E.g., images of nooses, pointing a gun)\n\n',
 '**Does the image include derogatory or threatening language, either in text form within the image or as part of the content displayed?** (E.g., captions suggesting harm or derogatory comments)\n\n',
 '**Does the image contain gestures or expressions commonly associated with disrespect or hostility toward individuals?** (E.g., inappropriate hand gestures, aggressive postures or facial expressions)\n',
 'Does the image include any gestures or symbols commonly associated with offensive or threatening behavior?\n',
 'Is the content of the image evidently aimed at disparaging or demeaning a specific individual or group based on their identity or personal attributes?\n',
 'Does the context or accompanying text of the image suggest it is being used to intimidate, harass, or provoke others?',
 'Does the image feature aggressive gestures or symbols that could intimidate or threaten an individual?\n',
 'Are there any elements or symbols in the image that convey harmful or threatening messages, such as depictions of violence or self-harm?\n',
 'Is the overall tone of the image derogatory, mocking, or demeaning towards individuals or groups?',
 'Does the image contain symbols, gestures, or visuals commonly associated with violence or threats (like guns pointed towards the viewer)?\n',
 'Does the image use humiliating, aggressive, or inappropriate gestures or expressions to belittle or frighten individuals?\n',
 'Does the context or caption of the image suggest encouragement of harm, fear, or distress towards an individual or group?',
 '**Does the image depict or contain symbols that express hostility or rejection towards specific individuals or groups?**\n   - This can include threatening gestures, offensive symbols, or explicit signs of disrespect or animosity.\n\n',
 '**Is the image accompanied by text or captions that could be interpreted as abusive, threatening, or excessively derogatory?**\n   - Contextual text that demeans or intimidates can transform a seemingly benign image into a vehicle for cyberbullying.\n\n',
 '**Does the content of the image clearly intend to embarrass, insult, or harm the reputation of an individual or group?**\n   - Images that are manipulated or those capturing individuals in compromising or demeaning scenarios can be used as a form of harassment or bullying.',
 '**Does the image include gestures or symbols that are commonly associated with offensive or threatening messages?**\n   - This refers to any visual gestures like middle fingers, gun signs, or other hand signals that could be considered aggressive or disrespectful.\n\n',
 '**Are there any texts or captions within the image that disparage or demean a specific individual or group?**\n   - Inspect for derogatory language, name-calling, or any text aimed at embarrassing or hurting another person or group.\n\n',
 "**Does the subject's expression or pose in the image suggest mockery, scorn, or intimidation towards others?**\n   - This includes facial expressions, body language, or actions that appear to mock, intimidate, or belittle others, which might be used to bully someone online.",
 'Does the image depict any gestures or expressions that could reasonably be interpreted as offensive or demeaning towards specific individuals or groups?\n',
 'Are there any text or symbols in the image that can be seen as threatening, harassing, or publicly shaming someone?\n',
 'Does the context or the setting of the image suggest a negative portrayal or mockery aimed at embarrassing or harming an individual’s reputation or well-being?',
 'Does the image contain symbols, gestures, or text that could be interpreted as offensive or harmful?\n',
 'Does the image depict or suggest violence, aggression, or the threat of harm towards an individual or group?\n',
 'Does the image make use of derogatory or demeaning language or imagery that targets specific characteristics such as race, gender, sexuality, or others?\n',
 'Does the image contain gestures or expressions commonly associated with offensive or threatening behavior?\n',
 'Are there any textual elements in the image that include insults, threats, or harmful language directed at an individual or group?\n',
 'Does the image depict any form of aggressive or violent behavior that could be interpreted as intimidating or causing harm?',
 '**Does the image depict gestures or symbols traditionally associated with insult or offense?** (e.g., middle-finger gestures)\n',
 '**Does the image contain elements that could be seen as promoting harm or violence against oneself or others?** (e.g., depictions of nooses or weapons aimed at the viewer)\n',
 '**Does the image include any text or imagery that might be designed to intimidate, threaten, or demean a person or group?** (e.g., aggressive facial expressions combined with offensive gestures)',
 'Does the image contain gestures or symbols that are universally recognized as offensive or threatening?\n',
 'Is the content of the image directly targeting or mocking a specific individual or group in a demeaning manner?\n',
 'Are there elements in the image, such as text overlays or context, that imply harm, ridicule, or exclusion towards others?',
 'Does the image depict or imply harm or violence, either physical or emotional, toward an individual or group?\n',
 'Does the expression or gesture in the image convey disapproval, disrespect, or ridicule that could be used to embarrass or demean someone?\n',
 'Is the context or associated text of the image intended to threaten, harass, or intimidate an individual or specific group?',
 'Does the image contain gestures or expressions that target or humiliate an individual or group?\n',
 'Are there visible signs or text within the image that could be perceived as threatening or harmful towards someone?\n',
 'Does the context or depiction in the image promote negative stereotypes or demeaning attitudes toward individuals based on their appearances or actions?',
 '**Does the image contain any text or symbols that are commonly associated with insults or threats?**  \n   (Assess if any overt messages of hostility, threats, or derogatory terms are visible.)\n\n',
 '**Does the image portray any gestures or expressions that could be interpreted as intimidating or mocking?**  \n   (Look for gestures like pointing a gun, thumbs down, or facial expressions that convey mockery or anger.)\n\n',
 '**Is the context or setting of the image implying a threatening or harassing tone towards an individual or group?**  \n   (Consider whether the backdrop or situation in the image might suggest a negative or harmful intent towards someone.)']


In [ ]:
!pip install -U sentence-transformers

In [ ]:
stops = ['.', '?']
filtered_criteria = set()

for stop in stops:
  for criteria in processed_criteria:
    print(criteria.split(stop))
    if len(criteria.split(stop)) == 1:
      filtered_criteria.add(criteria.replace('\n', ''))

In [ ]:
len(filtered_criteria)

In [ ]:
from sklearn.cluster import KMeans

from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

embedder = SentenceTransformer("all-mpnet-base-v2")

corpus = list(filtered_criteria)

corpus_embeddings = embedder.encode(corpus)


In [ ]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
)
t0 = time()
X_tfidf = vectorizer.fit_transform(corpus)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

In [ ]:
corpus_embeddings

In [ ]:
X_tfidf

In [ ]:
# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
clusters_k_means = []
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i + 1)
    print(cluster)
    clusters_k_means.append(cluster)
    print("")

In [ ]:
clustering_model = AgglomerativeClustering(
    n_clusters=None, distance_threshold=1.4
)  # , affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
clusters = []
for i, cluster in clustered_sentences.items():
    print("Cluster ", i + 1)
    print(cluster)
    clusters.append(cluster)
    print("")

In [ ]:
clustering_model = AgglomerativeClustering(
    n_clusters=None, distance_threshold=2
)  # , affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(X_tfidf.toarray())
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
clusters = []
for i, cluster in clustered_sentences.items():
    print("Cluster ", i + 1)
    print(cluster)
    clusters.append(cluster)
    print("")

In [ ]:
len(clusters)

In [ ]:
clustered_sentences

In [ ]:
clusters = [sorted(cluster, key = lambda x: len(x)) for cluster in clusters]
clusters[11]

In [ ]:
questions = ['Does the image contain symbols or gestures commonly associated with aggression or insults, such as middle fingers, guns pointed towards the viewer, an "L" gesture, or depictions of suicide?', 'Does the image depict or imply harm or threats towards an individual or a group?', 'Does the image include any text or symbols that could be interpreted to promote hostility towards specific groups or individuals?', 'Does the image contain gestures or expressions that are commonly associated with disrespect or hostility, such as middle fingers, sticking out tongues, or mockingly sad faces?', 'Is the context, content, or embedded text of the image suggest a scenario meant to embarrass, frighten, or demean another individual?', 'Are symbols of hate or exclusion prominently featured in the image?', 'Are there visual cues, such as the inclusion of weapons or potentially harmful objects, that could be interpreted as threatening or promoting harm?', 'Are there visual elements in the image that suggest harm or violence towards oneself or others?', 'Are there any elements in the image that specifically target, mock, or belittle someone based on their identity such as race, gender, sexuality, or personal beliefs?']

In [ ]:
questions

In [ ]:
def make_payload(base64_image, questions):
  payload = {
        "model": "gpt-4-turbo",
        "messages": [
          {
            "role": "user",
            "content": [

            ]
          }
        ],
        "max_tokens": 300
      }


  payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': 'Cyberbullying content is used to harass, threaten, embarrass, or target another person.  Does the following image contain cyberbullying content?\na. Yes\nb. No'
        }
    )

  payload['messages'][0]['content'].append(
        {
            'type': 'image_url',
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              'detail': 'low'
              }
        }
    )

  payload['messages'][0]['content'].append(
        {
            'type': 'text',
            'text': questions + '\nState your answer for each step, and then answer the question: Does the image contain cyberbullying content?\na. Yes\nb. No'
        }
    )

  return payload

In [ ]:
tp = 0
for image in train_set:
  image_path = f'{cyberbullying_path}/{image}'
  base64_image = encode_image(image_path)
  payload = make_payload(base64_image, '1. ' + questions[1])
  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  response_text = response.json()['choices'][0]['message']['content']
  print(payload)
  print(response_text)
  label = response_text_to_label(response_text)
  if label == 1:
    tp += 1
  print(label)
  print(tp)

In [ ]:
tp = 0
for image in train_set[20:]:
  image_path = f'{non_cyberbullying_path}/{image}'
  base64_image = encode_image(image_path)
  payload = make_payload(base64_image, '1. ' + questions[1])
  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  response_text = response.json()['choices'][0]['message']['content']
  print(payload)
  print(response_text)
  label = response_text_to_label(response_text)
  if label == 1:
    tp += 1
  print(label)
print(tp)

In [ ]:
final_set = [
    'Does the image contain symbols or gestures commonly associated with aggression or insults, such as middle fingers, guns pointed towards the viewer, an "L" gesture, or depictions of suicide?',
    'Are there visual cues, such as the inclusion of weapons or potentially harmful objects, that could be interpreted as threatening or promoting harm?',
    'Does the image include any text or symbols that could be interpreted to promote hostility towards specific groups or individuals?',
    'Are there any elements in the image that specifically target, mock, or belittle someone based on their identity such as race, gender, sexuality, or personal beliefs?'
    'Does the image depict or imply hostility or threats towards an individual or a group?'
]

In [ ]:
train_set = random.sample(cyberbullying_images, 20)
a = random.sample(non_cyberbullying_images, 5)
train_set += a

In [ ]:
len(questions)

In [ ]:
d = {'path': [], 'true_label': [], 'prompt': [], 'response': [], 'label': []}
df1 = pd.DataFrame(data=d)

In [ ]:
for image in tqdm(cyberbullying_images):
  payload = make_payload()
  try:
    image_path = f'{cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload['messages'][0]['content'].append(
      {
          'type': 'image_url',
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            'detail': 'low'
            }
      }
    )

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    response_text = response.json()['choices'][0]['message']['content']
    label = response_text_to_label(response_text)

    new_row = pd.DataFrame(data = {'path': [image_path], 'true_label': [1], 'prompt': [prompt], 'response': [response_text], 'label': [label]})

    df1 = pd.concat([df1, new_row], ignore_index=True)

    payload['messages'][0]['content'].pop(-1)
    #print(df1)

  except:
      print('Error: ')
      print(image)

In [ ]:
df1.to_csv('/content/drive/MyDrive/few_shot_cyberbullying.csv')

In [ ]:
d = {'path': [], 'true_label': [], 'prompt': [], 'response': [], 'label': []}
df = pd.DataFrame(data=d)

In [ ]:
for image in tqdm(non_cyberbullying_images):
  payload = make_payload()
  try:
    image_path = f'{non_cyberbullying_path}/{image}'
    base64_image = encode_image(image_path)

    payload['messages'][0]['content'].append(
      {
          'type': 'image_url',
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            'detail': 'low'
            }
      }
    )

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    response_text = response.json()['choices'][0]['message']['content']
    label = response_text_to_label(response_text)

    new_row = pd.DataFrame(data = {'path': [image_path], 'true_label': [0], 'prompt': [prompt], 'response': [response_text], 'label': [label]})

    df = pd.concat([df, new_row], ignore_index=True)

    payload['messages'][0]['content'].pop(-1)
    #print(df)

  except:
      print('Error: ')
      print(image)

In [ ]:
df.to_csv('/content/drive/MyDrive/few_shot_non_cyberbullying.csv')

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/few_shot_cyberbullying.csv')

In [ ]:
cyberbullying_results = [result for result in list(df1.loc[:,"label"]) if result != -1]
non_cyberbullying_results = [result for result in list(df.loc[:,"label"]) if result != -1]

In [ ]:
def get_stats(results, ground_truth):
  tp = tn = fp = fn = 0
  for result in results:
    if result:
      if ground_truth:
        tp += 1
      else:
        fp += 1

    else:
      if ground_truth:
        fn += 1

      else:
        tn += 1

  return len(results), tp, tn, fp, fn

In [ ]:
a = get_stats(cyberbullying_results, 1)

In [ ]:
b = get_stats(non_cyberbullying_results, 0)

In [ ]:
c = []
for i in range(5):
  c.append(a[i] + b[i])

In [ ]:
a

In [ ]:
b

In [ ]:
tp = c[1]
tn = c[2]
fp = c[3]
fn = c[4]


precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tp + tn) / (c[0])

In [ ]:
print(precision, recall, accuracy)